In [1]:
import pandas as pd
import re

In [2]:
df_stops = pd.read_csv("1964-stops-localised-final.csv")  

In [3]:
df_stops.drop(['stop_id'], axis=1, inplace=True)
df_stops['stop_name'] = df_stops['stop_name'].astype(str).str.strip()
df_stops['location'] = df_stops['location'].astype(str).str.strip()

In [4]:
df_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1444 entries, 0 to 1443
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   stop_name          1444 non-null   object
 1   type               1444 non-null   object
 2   in_lines           1444 non-null   object
 3   location           1444 non-null   object
 4   identifier         345 non-null    object
 5   stop_id_1961       970 non-null    object
 6   previous_in_lines  970 non-null    object
 7   stop_description   783 non-null    object
dtypes: object(8)
memory usage: 90.4+ KB


In [5]:
split_df = pd.concat([df_stops[col].astype(str).str.split(';', expand=True).stack().str.strip() for col in df_stops.columns], axis=1, keys=df_stops.columns)
df_stops = split_df.groupby(level=0).ffill().reset_index(drop=True)

In [6]:
df_stops = df_stops.fillna('').groupby(['stop_name', 'type', 'location']).agg({
    'in_lines': ', '.join,
    'identifier': ', '.join,
    'stop_description': ', '.join
}).reset_index()

# Reset the index
df_stops.reset_index(inplace=True)
df_stops.rename(columns={'index': 'stop_id'}, inplace=True)

df_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   stop_id           1456 non-null   int64 
 1   stop_name         1456 non-null   object
 2   type              1456 non-null   object
 3   location          1456 non-null   object
 4   in_lines          1456 non-null   object
 5   identifier        1456 non-null   object
 6   stop_description  1456 non-null   object
dtypes: int64(1), object(6)
memory usage: 79.8+ KB


In [7]:
df_line = pd.read_csv("../initial/line_data_1964.csv")  

In [8]:
for index, row in df_line.iterrows():
    # Check if the regex pattern (lowercase 's' or 'x' at the end) is found in the "in_lines" column
    if re.search(r'[sx]$', row["line_name"].lower()):
        # Delete the section called for in the regex
        df_line.at[index, "line_name"] = re.sub(r'[sx]$', '', row["line_name"])


In [9]:
# Replace U+00a0 with regular whitespace in the entire DataFrame
df_line.replace('\u00a0', ' ', regex=True, inplace=True)

In [10]:
df_line.head(2)

,line_name,Stops,Frequency,Length (time),type,east-west,year
0,1,Ostbahnhof - U-Bhf. Strausberger Platz - Lenin...,20.0,28.0,strassenbahn,Ost,1964
1,A1P,Adalbertstrasse - S-Bhf. Jannowitzbrücke - Ale...,10.0,6.0,bus,Ost,1964


In [11]:
# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
df_stops['stop_id'] = '1964' + df_stops['stop_id'].astype(str)
df_stops.head(2)

,stop_id,stop_name,type,location,in_lines,identifier,stop_description
0,19640,4. Ring,bus,"52.42250630689789, 13.2869490566549",A53,nan,4. Ring
1,19641,Adalbertstrasse,bus,"52.50548699996121, 13.42221404498021",A1P,nan,nan


In [12]:
df_line['line_name'] = df_line['line_name'].astype(str).str.strip()
df_line['Stops'] = df_line['Stops'].astype(str).str.strip()
df_line['year'] = df_line['year'].astype(int)
df_line['Frequency'] = pd.to_numeric(df_line['Frequency'], errors='coerce').fillna(0).astype(int)
df_line['Length (time)'] = pd.to_numeric(df_line['Length (time)'], errors='coerce').fillna(0).astype(int)

In [13]:
# Define a function to remove leftover parentheses from a column
def remove_double_whitespace(text):
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    text = text.replace("–", "-")
    text = text.replace(" - ", " - ")
    return text

# Apply the function to the 'line_stops' column
df_line['Stops'] = df_line['Stops'].apply(remove_double_whitespace)
df_line['Stops'] = df_line['Stops'].str.replace(r'\s*–\s*', ' - ', regex=True)

In [14]:
mask = df_line['Stops'].str.contains(" - ").fillna(True)
none_df = df_line[~mask]
none_df

,line_name,Stops,Frequency,Length (time),type,east-west,year


In [15]:
# Filter rows where 'line_stops' is not a string
non_string_rows = df_line[~df_line['Stops'].apply(lambda x: isinstance(x, str))]
non_string_rows

,line_name,Stops,Frequency,Length (time),type,east-west,year


In [16]:
### Creating related tables

In [17]:
def extract_first_start_stop(string):
    stations = string.split(" - ")
    first_station = stations[0]
    last_station = stations[-1]
    return f"{first_station}<> {last_station}"

In [18]:
def create_line_df(df):
    line_df = pd.DataFrame({
        'line_id': range(1, 1+len(df)),
        'year': df['year'],
        'line_name': df['line_name'],
        'type': df["type"],
        "start_stop": df['Stops'].apply(extract_first_start_stop),
        "Length (time)": df['Length (time)'],
        "east_west": df['east-west'],
        "Frequency": df['Frequency']
    })
    return line_df

In [19]:
line_df = create_line_df(df_line)

In [20]:
line_df.head(2)

,line_id,year,line_name,type,start_stop,Length (time),east_west,Frequency
0,1,1964,1,strassenbahn,Ostbahnhof<> Hannoversche Strasse,28,Ost,20
1,2,1964,A1P,bus,"Adalbertstrasse<> Alexanderplatz, Grunerstrasse",6,Ost,10


In [21]:
# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
line_df['line_id'] = '1964' + line_df['line_id'].astype(str)
line_df.head(2)

,line_id,year,line_name,type,start_stop,Length (time),east_west,Frequency
0,19641,1964,1,strassenbahn,Ostbahnhof<> Hannoversche Strasse,28,Ost,20
1,19642,1964,A1P,bus,"Adalbertstrasse<> Alexanderplatz, Grunerstrasse",6,Ost,10


In [22]:
def create_line_stops_df(df):
    line_stops = df['Stops'].str.split(' - ', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='stop_name')

    line_stops['stop_order'] = line_stops.groupby('index').cumcount()
    #index starts from 0 so it looks like 1 row is missing but this is not true

    # Clean the 'Stop Name' column by removing whitespace and non-breaking spaces
    line_stops['stop_name'] = line_stops['stop_name'].str.replace(u'\xa0', ' ').str.strip()

    # reset index so that it can be used for foreign key

    return line_stops

In [23]:
line_stops_df = create_line_stops_df(df_line)

In [24]:
for index, row in line_stops_df.iterrows():
    line_stops_df.at[index, "index"] = row["index"] + 1
line_stops_df.rename(columns={'index': 'line_id'}, inplace=True)

line_stops_df

,line_id,stop_name,stop_order
0,1,Ostbahnhof,0
1,1,U-Bhf. Strausberger Platz,1
2,1,Leninplatz,2
3,1,"Alexanderplatz, Memhardstr.",3
4,1,Rosenthaler Platz,4
...,...,...,...
2071,175,Magdalenenstr.,7
2072,175,Bahnhof Lichtenberg (Zentralfriedhof),8
2073,175,Friedrichsfelde (Tierpark),9
2074,176,Kladow,0


In [25]:
# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
line_stops_df['line_id'] = '1964' + line_stops_df['line_id'].astype(str)
line_stops_df.head(2)

,line_id,stop_name,stop_order
0,19641,Ostbahnhof,0
1,19641,U-Bhf. Strausberger Platz,1


In [26]:
# get a list of unique Line IDs in the line_stops DataFrame
line_ids = line_stops_df['line_id'].unique()

# loop through each unique Line ID
for line_id in line_ids:
    # get the first and last Stop Name for this Line ID
    first_stop_name = line_stops_df.loc[line_stops_df['line_id'] == line_id, 'stop_name'].iloc[0]
    last_stop_name = line_stops_df.loc[line_stops_df['line_id'] == line_id, 'stop_name'].iloc[-1]
    
    # get the corresponding Start-Stop value from line_df for this Line ID
    start_stop = line_df.loc[line_df['line_id'] == line_id, 'start_stop'].iloc[0]
    
    # check if the first and last Stop Names are contained in the Start-Stop value
    if first_stop_name not in start_stop or last_stop_name not in start_stop:
        print(f"Quality control check failed for line_id {line_id}.")

# I checked that this works using the following code:
# line_stops_df.loc[0, "stop_name"] = "test"
# and got the right result: Quality control check failed for line_id 1001.


In [27]:
line_df.head(3)

,line_id,year,line_name,type,start_stop,Length (time),east_west,Frequency
0,19641,1964,1,strassenbahn,Ostbahnhof<> Hannoversche Strasse,28,Ost,20
1,19642,1964,A1P,bus,"Adalbertstrasse<> Alexanderplatz, Grunerstrasse",6,Ost,10
2,19643,1964,3,strassenbahn,S-Bhf. Warschauer Strasse<> Björnsonstrasse,45,Ost,12


In [28]:
def add_type(line_stops, line_df):
    # Assuming line_id is the common column between line_stops and line_df
    merged_df = pd.merge(line_stops, line_df[['line_id', 'type', "line_name"]], on='line_id', how='left')
    
    # Rename the 'type' column from line_df to 'type_from_line_df' to avoid conflicts
    merged_df.rename(columns={'type': 'type'}, inplace=True)
    
    # Drop the 'type_from_line_df' column if it's not needed in the final result
    # merged_df.drop(columns=['type_from_line_df'], inplace=True)
    
    return merged_df

In [29]:
line_stops_df = add_type(line_stops_df, line_df)

In [30]:
line_stops_df.loc[120:130]

,line_id,stop_name,stop_order,type,line_name
120,196412,Kirschenallee,4,strassenbahn,53
121,196412,Schloss Ruhwald,5,strassenbahn,53
122,196412,U-Bhf. Ruhleben,6,strassenbahn,53
123,196412,Teltower Strasse,7,strassenbahn,53
124,196412,Freiheit,8,strassenbahn,53
125,196412,"Spandau, Markt",9,strassenbahn,53
126,196412,Hafenplatz,10,strassenbahn,53
127,196412,Lynastrasse Ecke Neuendorfer Strasse,11,strassenbahn,53
128,196412,Hohenzollernring,12,strassenbahn,53
129,196412,Hakenfelde,13,strassenbahn,53


In [31]:
df_stops

,stop_id,stop_name,type,location,in_lines,identifier,stop_description
0,19640,4. Ring,bus,"52.42250630689789, 13.2869490566549",A53,nan,4. Ring
1,19641,Adalbertstrasse,bus,"52.50548699996121, 13.42221404498021",A1P,nan,nan
2,19642,Adamstrasse,bus,"52.521029117218866, 13.19780699419913","A34,A35",nan,Adamstrasse Ecke Pichelsdorfer Strasse
3,19643,Adlershof,s-bahn,"52.434722222222,13.541388888889","KBS 106a,KBS 103,KBS 103a",Q323551,Berlin-Adlershof station
4,19644,Adolf-Scheidt-Platz,strassenbahn,"52.478194856980814, 13.37836902238447",96,nan,Adolf-Scheidt-Platz
...,...,...,...,...,...,...,...
1451,19641451,Zossener Strasse,bus,"52.49160676567746, 13.394443897793668",A28,nan,Zossener Strasse Ecke Gneisenaustrasse
1452,19641452,Zossener Strasse Ecke Blücherstrasse,strassenbahn,"52.49622054069677, 13.395106811633234",95,nan,nan
1453,19641453,Zum Heckeshorn,bus,"52.43307505069834, 13.164008736441371",A51,nan,Zum Heckeshorn Ecke Am Großen Wannsee
1454,19641454,Zweibrücker Strasse,bus,"52.54677409100436, 13.188764018875105",A63,nan,nan


In [32]:
def add_fk(line_stops_df, df_stops):
    # Create a new dataframe with the Stop Name and Stop ID columns
    stop_id_df = df_stops[['stop_name', 'stop_id', 'type', 'in_lines']]

    # Merge the line_stops_df and stop_id_df dataframes based on matching stop names and line_name condition
    line_stops_df = line_stops_df.merge(stop_id_df,
                                        left_on=['stop_name', 'type'],
                                        right_on=['stop_name', 'type'],
                                        how='left')

    # Filter the rows based on the condition that line_name is contained in in_lines
    line_stops_df = line_stops_df[line_stops_df.apply(lambda row: str(row['line_name']) in str(row['in_lines']), axis=1)]

    return line_stops_df

# Assuming line_stops_df and df_stops are your dataframes
# Replace 'stop_name', 'stop_id', 'type', and 'in_lines' with the actual column names you have

line_stops_df = add_fk(line_stops_df, df_stops)


In [33]:
# Calculate the difference between consecutive 'stop_order' values
line_stops_df['diff'] = line_stops_df['stop_order'].diff()

# Use the following line to drop rows where the 'diff' column is 0.0
line_stops_df = line_stops_df[line_stops_df['diff'] != 0.0]

# Identify faulty rows where the difference is not 1 digit behind
faulty_rows = line_stops_df[(line_stops_df['diff'] != 1) & (line_stops_df['stop_order'] != 0)]
faulty_rows

,line_id,stop_name,stop_order,type,line_name,stop_id,in_lines,diff
1427,1964116,"Lankwitz, Kirche",9,bus,A81,1964691,"A81,A2,A83,A32",2.0


In [34]:
faulty_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 1427 to 1427
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   line_id     1 non-null      object 
 1   stop_name   1 non-null      object 
 2   stop_order  1 non-null      int64  
 3   type        1 non-null      object 
 4   line_name   1 non-null      object 
 5   stop_id     1 non-null      object 
 6   in_lines    1 non-null      object 
 7   diff        1 non-null      float64
dtypes: float64(1), int64(1), object(6)
memory usage: 72.0+ bytes


In [35]:
# convert 'Stop ID' column to numeric values, coercing errors to NaN
line_stops_df['stop_id'] = pd.to_numeric(line_stops_df['stop_id'], errors='coerce')

# check if all values in 'Stop ID' column are numeric
if line_stops_df['stop_id'].notnull().all():
    print("All values in 'stop_id' column are numeric")
else:
    print("There are non-numeric values in 'stop_id' column")
    print(line_stops_df[line_stops_df['stop_id'].isnull()])


All values in 'stop_id' column are numeric


In [36]:
line_stops_df.drop(['line_name', "in_lines", "type", "diff"], axis=1, inplace=True)

In [37]:
line_stops_df.to_csv("line_stops_1964-final.csv")
line_df.to_csv("line_df_1964-final.csv")
df_stops.to_csv("stops_df_1964-final.csv")